# Практическое задание к уроку 5. Рекуррентные нейронные сети

1. Попробуйте обучить нейронную сеть LSTM на любом другом датасете (любимый временной ряд, текст на русском (другом языке) как генератор или классификатор, или прилагаемый набор airline-passengers - пасажиропоток для авиалиний). Опишите, какой результат вы получили? Что помогло вам улучшить ее точность?


2. *Попробуйте на numpy реализовать нейронную сеть архитектуры LSTM


3. *Предложите свои варианты решения проблемы исчезающего градиента в RNN

In [ ]:
!pip install --upgrade pip

In [ ]:
!pip install tensorflow -latest

In [ ]:
!pip install --upgrade tensorflow

In [ ]:
!pip install tensorflow_datasets

In [ ]:
# !pip install --upgrade google-api-python-client
# !pip install google-cloud

In [ ]:
# !pip install google-cloud-vision

In [ ]:
# !pip install google.cloud.bigquery

In [ ]:
# !pip install google.cloud.storage

In [ ]:
# !pip install google

In [ ]:
# !pip install google.colab
# !pip install google-colab

In [ ]:
import numpy as np
import pandas as pd

import matplotlib.pyplot as plt

from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, LSTM
from sklearn.preprocessing import MinMaxScaler

from google.colab import drive
drive.mount('drive')

In [ ]:
passengers_data = pd.read_csv("airline-passengers.csv")
passengers_data.head()

In [ ]:
plt.plot(passengers_data['Passengers'])

In [ ]:
dataset = passengers_data['Passengers']
dataset = np.array(dataset).reshape(-1,1)
dataset[:10]

In [ ]:
scaler = MinMaxScaler()
dataset = scaler.fit_transform(dataset)
dataset.min(),dataset.max()

In [ ]:
train_size = 100
test_size = 44
train = dataset[0:train_size, :]
train.shape

In [ ]:
test = dataset[train_size:144, :]
test.shape

In [ ]:
def get_data(dataset, look_back):
    dataX, dataY = [], []
    for i in range(len(dataset)-look_back-1):
        a = dataset[i:(i+look_back), 0]
        dataX.append(a)
        dataY.append(dataset[i+look_back, 0])
    return np.array(dataX), np.array(dataY)

In [ ]:
X_test, y_test = get_data(test, look_back)

In [ ]:
X_train = X_train.reshape(X_train.shape[0], X_train.shape[1], 1)
X_test = X_test.reshape(X_test.shape[0], X_test.shape[1], 1)

In [ ]:
X_train.shape

In [ ]:
model = Sequential()
model.add(LSTM(5, input_shape = (1, look_back)))
model.add(Dense(1))
model.compile(loss="mse", optimizer="adam", metrics=['mae'])

In [ ]:
model.summary()

In [ ]:
model.fit(X_train, y_train, epochs=25, batch_size=1)

In [ ]:
y_pred = model.predict(X_test)

In [ ]:
y_pred = scaler.inverse_transform(y_pred)
y_test = np.array(y_test)
y_test = y_test.reshape(-1, 1)
y_test = scaler.inverse_transform(y_test)

In [ ]:
plt.figure(figsize=(14,5))
plt.plot(y_test, label = 'Real')
plt.plot(y_pred, label = 'Predicted')
plt.ylabel('passengers')
plt.legend()
plt.show()

**Вывод:**
Судя по сдвигу на графике, предсказания запаздывают. Увеличение количества эпох и сокращение количества слоев до 2 способствовало улучшению результата.